# Explore new city
A Python script for creating visualizations from location history data.<br>
Full description on [GitHub](https://github.com/dmitryplaunov/explore-new-city). Copyright (c) 2020 Dmitry Plaunov; Licensed MIT.
<br><br><br>

## Importing CSV files
Importing CSV files that contain location history data (latitude and longitude coordinates).

In [ ]:
import re
from os import listdir
import pandas as pd

# a path to the folder with CSV files with location history
csv_folder = 'csv/'

# defining a regex name matching, if only some of the folder files need to be imported
pattern = re.compile('\.csv$') # right now we are matching all .csv files

# creating an empty list that will be populated with dataframes from each imported CSV file
all_df_list = []

# populating the above-mentioned list with imported CSV files that match the defined name pattern
for file in listdir(csv_folder):
    if re.search(pattern, file):
        df = pd.read_csv(csv_folder + file)
        all_df_list.append(df)

# merging all dataframes from the list into a single dataframe
# merging vertically (axis=0) and ignoring index of each individual dataframe
df = pd.concat(all_df_list, axis=0, ignore_index=True) 

print('Done!')

## Limiting location history to a specific area

Limiting location history only for the area that we are interested in, if the CSV files also contain data from other locations.<br>
The area is defined by boundaries in latitude and longitude, which can be checked for your location on [Open Street Map](https://www.openstreetmap.org/export#map).

In [ ]:
df = df[(df.lon<0) & (df.lon>-9.3) & (df.lat<39) & (df.lat>38)].reset_index(drop=True)

print('Done!')

## Printing out coordinates to be used to get an image of a map

You will need to import an image of a map that will be used in creating plots.<br>
In this code snippet we are printing out latitude and longitude boundaries of the location data that you have.<br>
You can use these boundaries to create an image of a map on [Open Street Map](https://www.openstreetmap.org/export#map).<br>
You can also slightly increase the numbers so that there is some space between the locations points and the edges of the map.

In [ ]:
data_boundary = ((round(df.lon.min(),3), round(df.lon.max(),3),      
                  round(df.lat.min(),3), round(df.lat.max(),3)))

print('Left: ', data_boundary[0], ', ','Right: ',data_boundary[1], ' (Longitude)', sep='')
print('Top: ',data_boundary[2], ', ', 'Bottom: ',data_boundary[3], ' (Latitude)', sep='')

## Importing an image of a map

In [ ]:
import matplotlib.pyplot as plt

# a path to the folder with an image of a map
map_folder = 'maps/'

# importing a map
map_image = plt.imread(map_folder + 'lisbon.jpg')

# specifying longitude and latitude boundaries of the map (can be viewed when sharing on OpenStreetMap)
image_boundary = (-9.2249, -9.1082, 38.6836, 38.7546) # (min_lon, max_lon, min_lat, max_lat)

# specifying the size of the map that was imported (can be viewed when sharing on OpenStreetMap)
image_width = 859
image_height = 670
aspect_ratio = image_width/image_height

# plots that use polygons require some adjustments 
image_boundary_a = (image_boundary[0],
                    image_boundary[1],
                    image_boundary[2]*aspect_ratio,
                    image_boundary[3]*aspect_ratio)

print('Done!')

## Defining breaking points for visualizations that show movement

Defining points in location history when the line needs to stop and start again from another place. This should happen, if the difference in time or location is too big (e.g. we shouldn't connect dots between the metro stations that a person took, since the area between them wasn't actually explored).

In [ ]:
from datetime import datetime, timedelta

# making sure that the date and time column has the right data type;
# if your dataset doesn't contain date and time data, ...
# ... you can skip this step and remove all references to time in this script
time_format = '%Y-%m-%d'
df.time = pd.to_datetime(df.time, format=time_format)

# creating a list with breaking points that will be appended with new values when the iteration below runs
breaks = [0]

# iterating through location history data points and defining when the line should break
for a in range(1,len(df.index)):
    
    # calculating the difference in time between the current and previous data points
    cur_time = df.iloc[[a]].time
    prev_time = df.iloc[[a-1]].time
    cur_time.index = prev_time.index # setting the same index to be able to compare dataframe values
    dif_time = cur_time - prev_time

    # calculating the difference in location between the current and previous data points
    cur_lat = df.iloc[[a]].lat
    cur_lon = df.iloc[[a]].lon
    prev_lat = df.iloc[[a-1]].lat
    prev_lon = df.iloc[[a-1]].lon
    cur_lat.index = prev_lat.index
    cur_lon.index = prev_lon.index
    dif_lat = cur_lat - prev_lat
    dif_lon = cur_lon - prev_lon
    distance = (dif_lat**2 + dif_lon**2)**1/2 # using the Pythagorean theorem (distance^2 = lat^2 + lon^2)
    
    # if the data points were recorded more than 1 hour or 1 km apart, ...
    # ... then add the current point to the breaking point list
    # using simplified calculation for the distance, assuming that the Earth is flat; 1km is 0.000083
    if dif_time.item() > timedelta(hours=1) or distance.item() > 0.000083:
        breaks.append(a)

print('Breaking points: ', breaks)

## Defining plot properties (colors, size, etc.)

In [ ]:
# map 'uncovering'
cover_color = '#404448' # hex color of the 'fog' over undiscovered parts, currently dark grey
cover_opacity = 0.85 # transparency of the 'fog', from 0 (transparent) to 1 (non-transparent)
dot_color = '#0F5C64' # hex color for the moving dot, currently turquois
dot_opacity = 1 # transparency of the moving dot, from 0 (transparent) to 1 (non-transparent)
dot_size = 20 # size of the moving dot
uncovered_size = 0.004 # size of the uncovered area around each data point

# heatmap
heatmap_alpha = 0.5 # transparency of the heatmap, from 0 (transparent) to 1 (non-transparent)
heatmap_sigma = 10 # amount of smoothing of the heatmap

# trace
line_color = '#008000' # hex color of the line, currently green
area_color = '#008000' # hex color of the area around the line, currently green
line_opacity = 1 # transparency of the line, from 0 (transparent) to 1 (non-transparent)
area_opacity = 0.5 # transparency of the area around the line, from 0 (transparent) to 1 (non-transparent)
covered_size = 0.004 # size of the covered area around each data point

print('Done!')

## Creating and exporting a plot for each data point

These plots can be then put together in a tool for creating gifs, specifying speed and repetition.

## Visualization 1. Map uncovering

In [ ]:
import numpy as np
from shapely.geometry import Point, LineString, box
from shapely.ops import cascaded_union, unary_union
import matplotlib.patches as ptc
from descartes import PolygonPatch
import gc

# a path to the folder where all the plots will be saved
output_folder = 'output/'

# instead of creating each plot from scratch, saving and then closing, ...
# we will create a figure before iterating through all data points ...
# and then inside the iteration loop just update the plot contents and save;
# this should help to reduce the script's memory usage

# creating a figure and an axis element on a single plot
# defining the resolution of the output ('figsize' multiplied by 'dpi')
fig, ax = plt.subplots(figsize=(8.6,6.7), dpi=166)

# if you have lots of data points, but limited memory on your device, ...
# you can define the end point ('counter_end') until which plots will be created.
# plots are created and exported during the script's running time, so nothing will be lost, ...
# if you suddenly get a memory issue.
counter = 0 # starting point
counter_end = 1000 # ending point 

# creating a plot for every entry in the location history dataframe
# you can add '[counter:counter_end]' at the end, if you have 'counter_end' defined
for moment in df.time.unique(): 
    
    # printing out a number every 10th plot, to indicate progress
    if counter % 10 == 0:
        print(counter)
    counter += 1
    
    # creating a dataframe consisting of all data points up to the current one
    sub_df = df[df.time <= moment]
    
    # initially creating a cover for the entire map
    cover = box(image_boundary_a[0], 
                image_boundary_a[2], 
                image_boundary_a[1], 
                image_boundary_a[3])
    covering_patch = ptc.Polygon(np.array(cover.exterior), facecolor="none")
    ax.add_patch(covering_patch)
    
    # drawing a single dot, representing the current exact location
    ax.scatter(sub_df.lon[counter-1], sub_df.lat[counter-1]*aspect_ratio, 
               zorder = 1,
               alpha = dot_opacity,
               c = dot_color,
               edgecolors = "none",
               s = dot_size)

    # creating a list of all data points up to the current one
    # this will be used to create 'uncovered' area
    points_list = []
    for i in range(len(sub_df.index)):
        data_point = Point(sub_df.iloc[[i]].lon.item(), sub_df.iloc[[i]].lat.item()*aspect_ratio)
        points_list.append(data_point)
    
    # creating a list that will be used for storing areas that were already 'uncovered'
    uncovered_area = []
    
    # for every line separated by breaking points
    # in the map 'uncovering' visualization adjacent 'uncovered' areas are also 'lines' 
    for breaking_point in range(1,len(breaks)):
        line_start = breaks[breaking_point-1]
        line_end = breaks[breaking_point]
         
        # 'uncovering' the map
        # if there is more than 1 data point, then draw a line ('lineal area')
        if len(points_list[line_start:line_end])>1:
            new_line = LineString(points_list[line_start:line_end])
            new = new_line.buffer(uncovered_size)
            uncovered_area.append(new)
            uncovered_polygons = cascaded_union(uncovered_area) # creating a union with previously 'uncovered' area, not overlapping
        
        # if there is only 1 data point, then only create a point ('rounded area')
        elif len(points_list[line_start:line_end]) == 1:
            only_point = points_list[0].buffer(uncovered_size)
            uncovered_area.append(only_point)
            uncovered_polygons = cascaded_union(uncovered_area)
    
    # only covering the area that consists of the previous cover, but excludes 'uncovered' areas
    covered_area = cover.difference(uncovered_polygons)
    new_covered_patch = PolygonPatch(covered_area, 
                                     fc=cover_color, 
                                     ec=cover_color, 
                                     alpha=cover_opacity, 
                                     zorder=1)
    ax.add_patch(new_covered_patch)

    # hiding axes
    plt.axis('off')
    
    # adding an image of a map behind all other elements (zorder is lower)
    ax.imshow(map_image, zorder=0, extent = image_boundary_a, aspect=1)
    
    # exporting the plot
    plt.savefig(output_folder + 'uncovering_map_' + str(counter) + '.jpg', bbox_inches = 'tight', pad_inches = 0)
    
    # clearing the plot
    plt.cla() 
    
    # deleting unwanted objects that are no longer in use to free the memory space
    gc.collect()

# completely closing the visualization
plt.close('all')

print('Done!')

## Visualization 2. Heatmap

In [ ]:
import matplotlib.cm as cm
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from matplotlib.colors import LinearSegmentedColormap

# a path to the folder where the heatmap will be saved
output_folder = 'output/'

# creating a figure and an axis element on a single plot
# defining the resolution of the output ('figsize' multiplied by 'dpi')
fig, ax = plt.subplots(figsize=(8.6,6.7), dpi=166)

# creating our custom colormap
# it will consist of the standard red for 'hot' areas, but transparent for 'cold' areas (not yellow or blue)

# defining the number of colors in the colormap
ncolors = 256
color_array = plt.get_cmap(cm.jet)(range(ncolors))

# changing opacity values for the first 64 colors in the colormap
# starting with '0.0' opacity (fully transparent) and ending with '1.0' (non-transparent)
color_array[0:64,-1] = np.linspace(0.0,1.0,64)

# creating a colormap object
map_object = LinearSegmentedColormap.from_list(name='my_cmap', colors = color_array) 

# registering this new colormap with matplotlib
plt.register_cmap(cmap=map_object)

# defining heatmap borders
df_heatmap_edges = pd.DataFrame({
    'lon':[image_boundary[0]-0.01, image_boundary[0]-0.01, image_boundary[1]+0.01, image_boundary[1]+0.01],
    'lat':[image_boundary[2]-0.01, image_boundary[3]+0.01, image_boundary[2]-0.01, image_boundary[3]+0.01]
})
df = df.append(df_heatmap_edges, ignore_index=True)

# generating a heatmap with 1076x839 data points (output file resolution)
heatmap, xedges, yedges = np.histogram2d(df.lon, df.lat, bins=(1076, 839))

# smoothing the heatmap
heatmap = gaussian_filter(heatmap, heatmap_sigma)

# getting the minimum and maximum lon and lat values
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]] 
ax.set_xlim(image_boundary[0],image_boundary[1])
ax.set_ylim(image_boundary[2],image_boundary[3])
plt.xlim([image_boundary[0],image_boundary[1]])
plt.ylim([image_boundary[2],image_boundary[3]])

# hiding axes
ax.axis('off')

# displaying the heatmap
ax.imshow(heatmap.T, origin='lower', extent=extent, cmap='my_cmap', zorder=1, alpha=heatmap_alpha)

# displaying the background image
plt.imshow(map_image, zorder=0, extent = image_boundary, aspect = aspect_ratio)

# exporting the image
plt.savefig(output_folder + '/heatmap_a' + str(heatmap_alpha)+ '_s' + str(heatmap_sigma) + '.jpg', bbox_inches = 'tight', pad_inches = 0)

# completely closing the visualization
plt.close()

print('Done!')

## Visualization 3. Trace

In [ ]:
from shapely.geometry import Point, LineString
from shapely.ops import cascaded_union, unary_union
import matplotlib.patches as ptc
from descartes import PolygonPatch
import gc

# a path to the folder where all the plots will be saved
output_folder = 'output/'

# instead of creating each plot from scratch, saving and then closing, ...
# we will create a figure before iterating through all data points ...
# and then inside the iteration loop just update the plot contents and save;
# this should help to reduce the script's memory usage

# creating a figure and an axis element on a single plot
# defining the resolution of the output ('figsize' multiplied by 'dpi')
fig, ax = plt.subplots(figsize=(8.6,6.7), dpi=166)

# if you have lots of data points, but limited memory on your device, ...
# you can define the end point ('counter_end') until which plots will be created.
# plots are created and exported during the script's running time, so nothing will be lost, ...
# if you suddenly get a memory issue.
counter = 0 # starting point
counter_end = 1000 # ending point
  
# creating a plot for every entry in the location history dataframe
# you can add '[counter:counter_end]' at the end, if you have 'counter_end' defined
for moment in df.time.unique():
    
    # printing out a number every 10th plot, to indicate progress
    if counter % 10 == 0:
        print(counter)
    counter += 1
    
    # creating a dataframe consisting of all data points up to the current one
    sub_df = df[df.time <= moment]
    
    # creating a list of all data points up to the current one
    # this will be used to create 'covered' area
    points_list = []
    for i in range(len(sub_df.index)):
        data_point = Point(sub_df.iloc[[i]].lon.item(), sub_df.iloc[[i]].lat.item()*aspect_ratio)
        points_list.append(data_point)
    
    # creating a list that will be used for storing areas that were already 'covered'
    covered_area = []
    
    # for every line separated by breaking points
    for breaking_point in range(1,len(breaks)):
        line_start = breaks[breaking_point-1]
        line_end = breaks[breaking_point]
        
        # drawing a line
        ax.plot(sub_df.lon[line_start:line_end],
                sub_df.lat[line_start:line_end]*aspect_ratio,
                zorder = 2,
                alpha = line_opacity,
                c = line_color,
                linewidth = 2)
        
        # 'covering' the map
        # if there is more than 1 point, then draw a line ('lineal area')
        if len(points_list[line_start:line_end])>1:
            new_line = LineString(points_list[line_start:line_end])
            new = new_line.buffer(covered_size)
            covered_area.append(new)
            covered_polygons = cascaded_union(covered_area) # creating a union with previously 'covered' area, not overlapping
        
        # if there is only 1 data point, then only create a point ('rounded area')    
        elif len(points_list[line_start:line_end]) == 1:
            only_point = points_list[0].buffer(covered_size)
            covered_area.append(only_point)
            covered_polygons = cascaded_union(covered_area)
            
    covered_polygon_patch = PolygonPatch(covered_polygons, 
                                         fc = area_color, 
                                         ec = area_color, 
                                         alpha = area_opacity, 
                                         zorder = 2)
    ax.add_patch(covered_polygon_patch)  
        
    # hiding axes
    plt.axis('off')
    
    # adding an image of a map behind all other elements (zorder is lower)
    plt.imshow(map_image, zorder=0, extent = image_boundary_a, aspect=1)
    
    # exporting the plot
    plt.savefig(output_folder + 'trace_map_' + str(counter) + '.jpg', bbox_inches = 'tight', pad_inches = 0)
    
    # clearing the plot
    plt.cla() 
    
    # deleting unwanted objects that are no longer in use to free the memory space
    gc.collect()
    
# completely closing the visualization
plt.close('all')

print('Done!')